In [6]:
%matplotlib inline
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
from keras.models import load_model
import glob
import h5py

Using TensorFlow backend.


In [7]:
from modules.factory import norm

In [8]:
rawfile = '../data/raw data/daily_futures.h5'
trainedfile = '../data/training set/40day_trend_labeled_futures.h5'
modelfile = '../data/keras models/40day-trend.h5'

In [9]:
period = 40

In [10]:
raw_file = h5py.File(rawfile, "r")
trained_file = h5py.File(trainedfile, "w")

In [11]:
model = load_model(modelfile)

In [28]:
'''
index N의 trend index는
N-period 에서 N-1 동안의 period 일에 대한 trend index임

'''

for idx, file in enumerate(raw_file.values()):
    name = file.attrs['name']
    print("\rprocessing %s...(%s/%s)"%(name, idx, len(raw_file)),end='', flush=True)
    columns = ['date', 'open','high','low','close','volume','open interest']
    raw_data = pd.DataFrame(file.value, columns=columns)
    trend=[np.nan for i in range(period)]
    
    for i in range(period, len(raw_data)):
        data = raw_data[i-period:i][['open','high','low','close']]
        data = norm(data)
        
        pred = model.predict(np.array([data.values]))
        trend.append(pred[0].argmax())
    raw_data['trend'] = pd.DataFrame(trend)
    raw_data = raw_data[period:]
    dset = trained_file.create_dataset(file.name, shape=raw_data.shape, data=raw_data.values)
    dset.attrs['name'] = file.attrs['name']
    dset.attrs['symbol'] = file.attrs['symbol']
    dset.attrs['exchange'] = file.attrs['exchange']
    dset.attrs['code'] = file.attrs['code']
    

processing CBOE VIX Futures...(0/75)
processing CME Australian Dollar AUD...(1/75)
processing CBOT Soybean Oil...(2/75)
processing CME British Pound GBP...(3/75)
processing CBOT Corn...(4/75)
processing CME Canadian Dollar CAD...(5/75)
processing NYMEX WTI Crude Oil...(6/75)
processing CME Class III Milk...(7/75)
processing CME Euro FX...(8/75)
processing CME Eurodollar...(9/75)
processing CME S&P 500 Index E-Mini...(10/75)
processing CBOT 30-day Federal Funds...(11/75)
processing CBOT 5-year US Treasury Note...(12/75)
processing NYMEX Gold...(13/75)
processing COMEX Copper...(14/75)
processing NYMEX Heating Oil...(15/75)
processing CME Japanese Yen JPY...(16/75)
processing CME Kansas City Wheat...(17/75)
processing CME Random Length Lumber...(18/75)
processing CME Live Cattle...(19/75)
processing CME Lean Hogs...(20/75)
processing CME S&P 400 Midcap Index...(21/75)
processing CME Mexican Peso...(22/75)
processing CME New Zealand Dollar NZD...(23/75)
processing NYMEX Natural Gas...(24/

In [29]:
raw_file.close()
trained_file.close()

In [30]:
from collections import Counter

In [32]:
f = h5py.File(trainedfile, "r")

In [38]:
trend = np.array([])
for file in f.values():
    trend = np.append(trend, file.value[:,4])

In [42]:
Counter(trend)

Counter({0.0: 30492, 1.0: 105057, 2.0: 47026})

In [27]:
v.attrs['symbol']

'None'